# Tubes Mata Kuliah Kecerdasan Buatan

## kelompok 2 : "Prediksi Prestasi Akademik Siswa Menggunakan Machine Learning Berdasarkan Faktor Demografis dan Kehadiran"

Link Dataset: https://www.kaggle.com/datasets/waqi786/student-performance-dataset/data

Anggota :
1. Rakasya Yoga
2. R. Rafi Yudi Pramana
3. Datu reksa Hamza Putra
4. M. Ali Abdillah Khotami
5. Aditya

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

Kode di atas digunakan untuk mengimport library-library yang dibutuhkan, seperti numpy untuk perhitungan numerik (array), pandas untuk dataframe, plt dan sns untuk visualisasi, dan sklearn untuk modelling

In [2]:
# df = pd.read_csv("../dataset/student_performance_data2.csv")
# df.head()

import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Load Data
df = pd.read_csv("../dataset/student-por.csv")  # atau student-por.csv

# Label encoding untuk fitur biner
binary_cols = [
    "school", "sex", "address", "famsize", "Pstatus",
    "schoolsup", "famsup", "paid", "activities", "nursery",
    "higher", "internet", "romantic"
]

le = LabelEncoder()
for col in binary_cols:
    df[col] = le.fit_transform(df[col])

# One-Hot encoding untuk fitur nominal multikategori
multi_cols = ["Mjob", "Fjob", "reason", "guardian"]
df = pd.get_dummies(df, columns=multi_cols, drop_first=True)

# Target Encoding (ubah G3 → kelas kategorikal)
def convert_grade(val):
    if val >= 16: return "A"
    elif val >= 14: return "B"
    elif val >= 10: return "C"
    else: return "D"

df["G3_Class"] = df["G3"].apply(convert_grade)

# Drop nilai mentah G3 jika hanya prediksi kelas
df = df.drop(columns=["G3"])

In [4]:
df.head()

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_health,Fjob_other,Fjob_services,Fjob_teacher,reason_home,reason_other,reason_reputation,guardian_mother,guardian_other,G3_Class
0,0,0,18,1,0,0,4,4,2,2,...,False,False,False,True,False,False,False,True,False,C
1,0,0,17,1,0,1,1,1,1,2,...,False,True,False,False,False,False,False,False,False,C
2,0,0,15,1,1,1,1,1,1,2,...,False,True,False,False,False,True,False,True,False,C
3,0,0,15,1,0,1,4,2,1,3,...,False,False,True,False,True,False,False,True,False,B
4,0,0,16,1,0,1,3,3,1,2,...,False,True,False,False,True,False,False,False,False,C


Kode di atas digunakan untuk membaca dataset yang akan digunakan, yaitu student_performance_data.csv. Kemudian 5 data teratas ditampilkan menggunakan head.

dalam data tersebut terdapat Kolom-kolom sebagai berikut:

1. sex: Jenis kelamin mahasiswa (0 = perempuan, 1 = laki-laki)  
2. Age: Usia mahasiswa  
3. StudyHoursPerWeek: Jumlah jam belajar per minggu  
4. AttendanceRate: Persentase kehadiran mahasiswa di kelas  
5. PartTimeJob: Status pekerjaan paruh waktu (0 = tidak bekerja, 1 = bekerja)  
6. ExtraCurricularActivities: Keterlibatan dalam kegiatan ekstrakurikuler (0 = tidak ikut, 1 = ikut)
7. GPA: Indeks prestasi kumulatif mahasiswa  



## Data Exploration

In [7]:
# FEATURE SELECTION
X = df.drop("G3_Class", axis=1)  # hapus features
y = df["G3_Class"]  # kelas label

# DATA SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [8]:
# MODEL TRAINING
# Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)
pred_nb = model_nb.predict(X_test_scaled)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_test)

# SVM
model_svm = SVC(kernel='rbf', C=1, gamma='scale')
model_svm.fit(X_train_scaled, y_train)
pred_svm = model_svm.predict(X_test_scaled)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)


## MODEL PERFORMANCE COMPARISON

In [9]:
# EVALUATION
models = {
    "Naive Bayes": pred_nb,
    "Decision Tree": pred_dt,
    "SVM": pred_svm,
    "Random Forest": pred_rf
}

print("\n====== MODEL PERFORMANCE COMPARISON ======\n")
for name, pred in models.items():
    print(f"📌 {name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))
    print("-" * 50)

# OPTIONAL: CONFUSION MATRIX
print("\nConfusion Matrices:")
for name, pred in models.items():
    cm = confusion_matrix(y_test, pred)
    print(f"\n{name}:\n{cm}")


====== MODEL PERFORMANCE COMPARISON ======

📌 Naive Bayes
Accuracy: 0.4308
              precision    recall  f1-score   support

           A       0.28      1.00      0.44        13
           B       0.00      0.00      0.00        12
           C       0.91      0.30      0.45        33
           D       0.71      0.71      0.71         7

    accuracy                           0.43        65
   macro avg       0.48      0.50      0.40        65
weighted avg       0.59      0.43      0.40        65

--------------------------------------------------
📌 Decision Tree
Accuracy: 0.7231
              precision    recall  f1-score   support

           A       0.86      0.92      0.89        13
           B       0.42      0.42      0.42        12
           C       0.78      0.76      0.77        33
           D       0.71      0.71      0.71         7

    accuracy                           0.72        65
   macro avg       0.69      0.70      0.70        65
weighted avg       0.72  

In [10]:
from sklearn.feature_selection import mutual_info_classif

info = mutual_info_classif(X, y, random_state=1)

info_fitur = pd.DataFrame({"Fitur": X.columns, "Informasi": info})
info_fitur = info_fitur.sort_values(by="Informasi")

print(info_fitur)

                Fitur  Informasi
3             address   0.000000
15            nursery   0.000000
13               paid   0.000000
20           freetime   0.000000
29         Mjob_other   0.000000
28        Mjob_health   0.000000
33         Fjob_other   0.000000
32        Fjob_health   0.000000
36        reason_home   0.000000
34      Fjob_services   0.000000
39    guardian_mother   0.000000
14         activities   0.004386
19             famrel   0.005115
30      Mjob_services   0.006004
8          traveltime   0.006149
5             Pstatus   0.007561
25           absences   0.007976
2                 age   0.008301
9           studytime   0.008694
35       Fjob_teacher   0.010663
22               Dalc   0.014253
11          schoolsup   0.016030
12             famsup   0.016323
40     guardian_other   0.016543
4             famsize   0.017310
18           romantic   0.018157
24             health   0.018717
37       reason_other   0.019419
1                 sex   0.024090
7         

## Feature Selection

In [33]:
# FEATURE SELECTION
cols_to_drop = ["address", "nursery", "paid", "freetime", "Mjob_other", "Mjob_health","Fjob_other", 
                "Fjob_health", "reason_home", "Fjob_services", "guardian_mother", "G3_Class"]
                # "sex", "Pstatus", "famsup", "studytime", "activities", "internet", "Dalc", "Walc", "Mjob_services",
                # "Mjob_teacher", "Fjob_teacher", "reason_other", "guardian_other", "absences", "Medu", "famsize", "health", "romantic"]

X = df.drop(columns=cols_to_drop, axis=1)

# DATA SPLIT
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.1, random_state=42
)

# FEATURE SCALING
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [35]:
# X.head()
# from sklearn.feature_selection import mutual_info_classif

# info = mutual_info_classif(X, y, random_state=1)

# info_fitur = pd.DataFrame({"Fitur": X.columns, "Informasi": info})
# info_fitur = info_fitur.sort_values(by="Informasi")

# print(info_fitur)

In [36]:
# MODEL TRAINING
# Naive Bayes
model_nb = GaussianNB()
model_nb.fit(X_train_scaled, y_train)
pred_nb = model_nb.predict(X_test_scaled)

# Decision Tree
model_dt = DecisionTreeClassifier(random_state=42)
model_dt.fit(X_train, y_train)
pred_dt = model_dt.predict(X_test)

# SVM
model_svm = SVC(kernel='rbf', C=1, gamma='scale')
model_svm.fit(X_train_scaled, y_train)
pred_svm = model_svm.predict(X_test_scaled)

# Random Forest
model_rf = RandomForestClassifier(n_estimators=150, random_state=42)
model_rf.fit(X_train, y_train)
pred_rf = model_rf.predict(X_test)

# EVALUATION
models = {
    "Naive Bayes": pred_nb,
    "Decision Tree": pred_dt,
    "SVM": pred_svm,
    "Random Forest": pred_rf
}

print("\n====== MODEL PERFORMANCE COMPARISON ======\n")
for name, pred in models.items():
    print(f"📌 {name}")
    print("Accuracy:", round(accuracy_score(y_test, pred), 4))
    print(classification_report(y_test, pred))
    print("-" * 50)

# OPTIONAL: CONFUSION MATRIX
print("\nConfusion Matrices:")
for name, pred in models.items():
    cm = confusion_matrix(y_test, pred)
    print(f"\n{name}:\n{cm}")


====== MODEL PERFORMANCE COMPARISON ======

📌 Naive Bayes
Accuracy: 0.4308
              precision    recall  f1-score   support

           A       0.30      1.00      0.46        13
           B       0.00      0.00      0.00        12
           C       0.83      0.30      0.44        33
           D       0.71      0.71      0.71         7

    accuracy                           0.43        65
   macro avg       0.46      0.50      0.40        65
weighted avg       0.56      0.43      0.39        65

--------------------------------------------------
📌 Decision Tree
Accuracy: 0.7692
              precision    recall  f1-score   support

           A       0.91      0.77      0.83        13
           B       0.50      0.58      0.54        12
           C       0.82      0.85      0.84        33
           D       0.83      0.71      0.77         7

    accuracy                           0.77        65
   macro avg       0.77      0.73      0.74        65
weighted avg       0.78  

In [58]:
# print("Naive Bayes")
# print("Accuracy:", round(accuracy_score(y_test, pred_nb), 4))
# print(classification_report(y_test, pred_nb))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_nb)
# print("Naive Bayes")
# print(cm)

# print("Random Forest")
# print("Accuracy:", round(accuracy_score(y_test, pred_rf), 4))
# print(classification_report(y_test, pred_rf))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_rf)
# print("Random Forest")
# print(cm)

# print("SVM")
# print("Accuracy:", round(accuracy_score(y_test, pred_svm), 4))
# print(classification_report(y_test, pred_svm))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_svm)
# print("SVM")
# print(cm)

# print("Decision Tree")
# print("Accuracy:", round(accuracy_score(y_test, pred_dt), 4))
# print(classification_report(y_test, pred_dt))
# print("-" * 50)

# # CONFUSION MATRIX
# print("\nConfusion Matrices:")
# cm = confusion_matrix(y_test, pred_dt)
# print("Decision Tree")
# print(cm)